In [4]:
import torch
!pip install captum
from torch import nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from captum.attr import IntegratedGradients

# Load the saved model
classification_model = torch.load('cats_vs_dogs_model.pth')
classification_model.eval()

# Define image preprocessing
image_preprocessing = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

# Function to transform tensor to an image
def tensor_to_image(tensor_img):
    img = tensor_img.permute(1, 2, 0).detach().numpy()
    img = np.clip(img, 0, 1)
    return img

# Load the raw image
raw_image = Image.open('cat_or_dog.jpg')

# Preprocess the image
processed_image = image_preprocessing(raw_image).unsqueeze(0)

# Function to display attributions
def display_attributions(attr_map, processed_image):
    plt.imshow(tensor_to_image(processed_image), alpha=0.5)
    plt.imshow(attr_map.squeeze(0).permute(1, 2, 0).detach().numpy(), cmap='hot', alpha=0.5)
    plt.show()

# Initialize Integrated Gradients
integrated_gradients = IntegratedGradients(classification_model)

# Set the baseline (black image) for attribution
black_image = torch.zeros_like(processed_image)

# Generate attribution map with Integrated Gradients
attr_map, convergence_metric = integrated_gradients.attribute(processed_image, black_image, return_convergence_delta=True)

# Display the attributions
display_attributions(attr_map, processed_image)

# Print the convergence delta for verification
print("Convergence Delta:", convergence_metric)



<ipython-input-4-bdfea13c229b>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classification_model = torch.load('cats_vs_dogs_model.pth')


FileNotFoundError: [Errno 2] No such file or directory: 'cats_vs_dogs_model.pth'